In [1]:
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu121
!pip install --no-deps packaging ninja einops flash-attn trl peft accelerate bitsandbytes
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.

In [2]:
import torch
import os
from google.colab import userdata
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments,GemmaTokenizer
from peft import LoraConfig
from trl import SFTTrainer

In [3]:
import os
os.environ["HF_TOKEN"]=userdata.get("HF_TOKEN")

In [4]:
model_id="google/gemma-2b"
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
tokenizer=AutoTokenizer.from_pretrained(model_id,use_auth_token=True)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [6]:
model=AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0},
    token=os.environ["HF_TOKEN"]
)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [9]:
text="Quote: imagination is more,"
device="cuda:0"
input=tokenizer(text,return_tensors="pt").to(device)
output=model.generate(**input,max_new_tokens=50)

In [10]:
answer=tokenizer.decode(output[0],skip_special_tokens=True)

In [11]:
answer

'Quote: imagination is more, than the sum of its parts.\n\nThe first thing you need to know about me is that I am a creative. I love to create. I love to design. I love to build. I love to write. I love to paint. I'

In [12]:
os.environ["WANDB_DISABLED"]="false"
lora_config=LoraConfig(
    r=8,
    target_modules=["q_proj","o_proj","k_proj","v_proj","gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM"
)

In [13]:
from datasets import load_dataset
data=load_dataset("Abirate/english_quotes")
data=data.map(lambda samples:tokenizer(samples["quote"]),batched=True)

README.md:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [14]:
def format_func(example):
  text=f"Quote: {example['quote'][0]}\nAuthor: {example['author'][0]}"
  return [text]

In [15]:
import transformers
trainer = SFTTrainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=format_func
)

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:382: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a formatting function. Therefore `formatting_func` will be ignored. Either remove the `formatting_func` or pass a dataset that is not already processed.
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/2508 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [16]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: manumanuvkm123 (manumanuvkm123-toch-institute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
1,2.560100
2,1.627000
3,2.479700
4,2.751500
5,2.298500
6,2.473000
7,2.877300
8,2.233200
9,3.176100
10,2.207300


TrainOutput(global_step=100, training_loss=2.055391828417778, metrics={'train_runtime': 140.4789, 'train_samples_per_second': 2.847, 'train_steps_per_second': 0.712, 'total_flos': 189744345784320.0, 'train_loss': 2.055391828417778})

In [27]:
text="Quote: a women like a teabag,"
device="cuda:0"
input=tokenizer(text,return_tensors="pt").to(device)
output=model.generate(**input,max_new_tokens=50)

In [28]:
print(tokenizer.decode(output[0],skip_special_tokens=True))

Quote: a women like a teabag, you never know how strong she is until she's in hot water.

I'm not sure if this is a quote or not, but I've heard it before. I'm not sure if it's a quote or not
